# Random Forest  + 'features'

## Train set

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
import cPickle
import warnings
warnings.filterwarnings("ignore")
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import chi2
from sklearn.cross_validation import KFold

pd.options.display.max_columns = 160

In [2]:
# all_df_original = pd.read_json('train.json')
all_df = pd.read_csv('train_feats_max_desc.csv')

In [3]:
all_df.shape

(49352, 83)

In [4]:
# all_df.head(2)

In [5]:
x_train, x_val, y_train, y_val = train_test_split(all_df.drop(['interest_level'], 1),all_df[['interest_level']], test_size=0.2, random_state=42)


In [6]:
cat_feats = cPickle.load(open('cat_feats.p', 'rb'))

for col in ['interest_level']:
    y_train[col] = y_train[col].astype('category')
    y_val[col] = y_val[col].astype('category')
    
for col in cat_feats:
    x_train[col] = x_train[col].astype('category')
    x_val[col] = x_val[col].astype('category')

In [7]:
drop_list = [u'listing_id', 'index']
x_train_small = x_train.drop(drop_list,1)

x_val_small = x_val.drop(drop_list,1)

In [8]:
# x_train_small_nolon = x_train_small.drop(['longitude'], axis=1)
# scores, pvalues = chi2(x_train_small_nolon, y_train)

In [9]:
# low_pvalues_cols = x_train_small_nolon.columns[(pvalues < 0.05)]

In [10]:
# low_pvalues_cols

In [11]:
# low_pvalues_cols = cPickle.load(open('low_pvalues_cols.p', 'rb'))
x_train_best = x_train_small#[low_pvalues_cols]
x_val_best = x_val_small#[low_pvalues_cols]
x_train_best.shape
# x_train_best.head(2)

(39481, 80)

In [9]:
# C_range = np.logspace(-2, 10, 13)
# gamma_range = np.logspace(-9, 3, 13)
# param_grid = dict(gamma=gamma_range, C=C_range)
# cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
# grid.fit(x_train_best[:10], y_train[:10])

# print("The best parameters are %s with a score of %0.2f"
#       % (grid.best_params_, grid.best_score_))

# # Now we need to fit a classifier for all parameters in the 2d version
# # (we use a smaller set of parameters here because it takes a while to train)

# C_2d_range = [1e-2, 1, 1e2]
# gamma_2d_range = [1e-1, 1, 1e1]
# classifiers = []
# for C in C_2d_range:
#     for gamma in gamma_2d_range:
#         clf = SVC(C=C, gamma=gamma)
#         clf.fit(X_2d, y_2d)
#         classifiers.append((C, gamma, clf))

In [27]:
# model = RandomForestClassifier()#n_estimators=10, random_state=0)
# parameters = {'max_features':[0.3, 0.5], 'max_depth':[10, 20], 'min_samples_split':[5, 10]}
# clf = GridSearchCV(model, parameters, scoring='log_loss', cv=2)
# # alphas = np.array([0.3,0.5])
# # grid = GridSearchCV(estimator=model, param_grid=dict(max_features=alphas))


In [28]:
# grid.fit(x_train_best, y_train)

In [18]:
model = RandomForestClassifier(n_estimators=1000, max_features=0.5, max_depth=20, min_samples_split=20, random_state=0, n_jobs=-1)
model = model.fit(x_train_best, y_train)

In [19]:
predicted_train = pd.DataFrame(model.predict_proba(x_train_best))
# predicted = model.predict_proba(x)
predicted_train.columns = ['high', 'low', 'medium']
predicted_train.head()
# predicted

,high,low,medium
0,0.021474,0.891110,0.087416
1,0.002449,0.966434,0.031117
2,0.054558,0.571732,0.373710
3,0.083598,0.723263,0.193139
4,0.024353,0.864821,0.110826


In [20]:
log_loss_train = log_loss(y_train, predicted_train.as_matrix())
log_loss_train

0.40162879184380418

In [21]:
predicted_val = pd.DataFrame(model.predict_proba(x_val_best))
# predicted = model.predict_proba(x)
predicted_val.columns = ['high', 'low', 'medium']
predicted_val.head()
# predicted

,high,low,medium
0,0.063540,0.688537,0.247922
1,0.084760,0.545504,0.369736
2,0.010658,0.863525,0.125817
3,0.037549,0.785394,0.177057
4,0.033296,0.779291,0.187413


In [22]:
log_loss_val = log_loss(y_val, predicted_val.as_matrix())
log_loss_val
# 0.68867588674963398, 0.68 ok (n_estimators=20, max_features=0.5, max_depth=5, min_samples_split=2, random_state=0)
# 0.65246470021282676, 0.60 ok (n_estimators=20, max_features=0.5, max_depth=5, min_samples_split=2, random_state=0)
# 0.65992553205901083, 0.47953531208509781 OF (n_estimators=100, max_features='sqrt', max_depth=20, min_samples_split=2, random_state=0) 
# 0.64955018062340741, 0.5490683949104378 OF (n_estimators=100, max_features='sqrt', max_depth=20, min_samples_split=10, random_state=0)

# new
# best_loss: 0.627188125099
# max_features: 0.3; max_depth: 20; min_samples_split: 20
# 0.60709048796069809 (n_estimators=1000, max_features=0.5, max_depth=20, min_samples_split=20

0.60709048796069809

In [23]:
# imp = zip(x_train_best.columns, list(model.feature_importances_))
# imp.sort(key = lambda t: t[1], reverse=True)

In [12]:
# bad_feats = [i[0] for i in imp[-20:]]
bad_feats = cPickle.load(open('bad_feats.p', 'rb'))


In [13]:
x_train_best.drop(bad_feats, axis=1, inplace=True)
x_val_best.drop(bad_feats, axis=1, inplace=True)

In [14]:
x_train_best.shape, x_val_best.shape

((39481, 60), (9871, 60))

## Cross Validation

In [15]:
def get_cv_loss(mf, md, mss):
    kf = KFold(x_train_best.shape[0], n_folds=5, random_state=2017)
    loss_tr = []
    loss_ts = []
    for train_index, test_index in kf:
        x_tr = x_train_best.reset_index().loc[train_index].set_index(['index'])
        y_tr =  y_train.reset_index().loc[train_index].set_index(['index'])
        x_ts = x_train_best.reset_index().loc[test_index].set_index(['index'])
        y_ts = y_train.reset_index().loc[test_index].set_index(['index'])

        model = RandomForestClassifier(n_estimators=1000, max_features=mf, max_depth=md, min_samples_split=mss, random_state=23, n_jobs=-1)
        model = model.fit(x_tr, y_tr)

        predicted_ts = pd.DataFrame(model.predict_proba(x_ts))
        predicted_ts.columns = ['high', 'low', 'medium']
        log_loss_ts = log_loss(y_ts, predicted_ts.as_matrix())
        loss_ts.append(log_loss_ts)
    
    print 'CV loss:', np.mean(loss_ts)
    predicted_val = pd.DataFrame(model.predict_proba(x_val_best))
    predicted_val.columns = ['high', 'low', 'medium']
    log_loss_val = log_loss(y_val, predicted_val.as_matrix())
    print 'Val loss:', log_loss_val
    return np.mean(loss_ts)

In [17]:
max_features = [0.3, 0.4, 0.5] #'sqrt',0.2, 
max_depth = [10, 12, 14, 16, 18, 20]
min_samples_split = [2, 5, 10, 20]
best_loss = 100
best_params = ''
count = 49
for mf in max_features:
    for md in max_depth:
        for mss in min_samples_split:
            print count
            count += 1
            params = 'max_features: ' + str(mf) + '; max_depth: ' + str(md) + '; min_samples_split: ' + str(mss)
            print params
            if mf == 0.3:
                if md in (10, 12, 14):
                    continue

#             model = RandomForestClassifier(n_estimators=500, max_features=mf, max_depth=md, min_samples_split=mss, random_state=0)
#             model = model.fit(x_train_best, y_train)
#             predicted_train = pd.DataFrame(model.predict_proba(x_train_best))
#             predicted_train.columns = ['high', 'low', 'medium']
#             log_loss_train = log_loss(y_train, predicted_train.as_matrix())
#             print 'Train loss:', log_loss_train
#             predicted_val = pd.DataFrame(model.predict_proba(x_val_best))
#             predicted_val.columns = ['high', 'low', 'medium']
#             log_loss_val = log_loss(y_val, predicted_val.as_matrix())
#             print 'Val loss:', log_loss_val
#             if log_loss_val < best_loss:
#                 best_loss = log_loss_val
#                 best_params = params
            cv_loss = get_cv_loss(mf, md, mss)
            if cv_loss < best_loss:
                best_loss = cv_loss
                best_params = params   
                
print 'best_loss:', best_loss
print 'best_params:', best_params
# best_loss: 0.638016960424
# best_params: max_features: 0.3; max_depth: 18; min_samples_split: 20

49
max_features: 0.3; max_depth: 10; min_samples_split: 2
50
max_features: 0.3; max_depth: 10; min_samples_split: 5
51
max_features: 0.3; max_depth: 10; min_samples_split: 10
52
max_features: 0.3; max_depth: 10; min_samples_split: 20
53
max_features: 0.3; max_depth: 12; min_samples_split: 2
54
max_features: 0.3; max_depth: 12; min_samples_split: 5
55
max_features: 0.3; max_depth: 12; min_samples_split: 10
56
max_features: 0.3; max_depth: 12; min_samples_split: 20
57
max_features: 0.3; max_depth: 14; min_samples_split: 2
58
max_features: 0.3; max_depth: 14; min_samples_split: 5
59
max_features: 0.3; max_depth: 14; min_samples_split: 10
60
max_features: 0.3; max_depth: 14; min_samples_split: 20
61
max_features: 0.3; max_depth: 16; min_samples_split: 2
CV loss: 0.615276730368
Val loss: 0.616430655878
62
max_features: 0.3; max_depth: 16; min_samples_split: 5
CV loss: 0.615071436554
Val loss: 0.616783622836
63
max_features: 0.3; max_depth: 16; min_samples_split: 10
CV loss: 0.615318231003
V



## Competition Test Set

In [26]:
test_df = pd.read_csv('test_feats_max_desc.csv')

In [27]:
test_df.head(2)

,index,bathrooms,bedrooms,created,latitude,listing_id,longitude,price,num_photos,elevator,hardwood floors,doorman,dishwasher,laundry in building,no fee,fitness center,pre-war,roof deck,outdoor space,dining room,high speed internet,balcony,swimming pool,new construction,terrace,exclusive,loft,garden/patio,wheelchair access,fireplace,simplex,lowrise,garage,reduced fee,furnished,multi-level,high ceilings,private outdoor space,parking space,live in super,renovated,green building,storage,stainless steel appliances,light,granite kitchen,bike room,exposed brick,marble bath,pets on approval,walk in closet(s),valet,subway,residents lounge,highrise,short term allowed,childrens playroom,no pets,duplex,actual apt. photos,central a/c,view,live/work,virtual doorman,sauna,microwave,shares ok,post-war,brownstone,business center,sublet,midrise,pet friendly,guarantors accepted,attended lobby,package room,video intercom,community recreation facilities,skylight,flex-2,cable/satellite tv,all utilities included
0,0,1.0,1,1.465623e+09,40.7185,7142618,-73.9865,2950,8,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1.0,2,1.466750e+09,40.7278,7210040,-74.0000,2850,3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [28]:
x_test = test_df[x_train_best.columns]
x_test.shape

(74659, 60)

In [29]:
pred_x = pd.DataFrame(model.predict_proba(x_test))
pred_x.columns = ['high', 'low', 'medium']
pred_x.head()

,high,low,medium
0,0.024834,0.711241,0.263924
1,0.257538,0.342242,0.400220
2,0.015254,0.897701,0.087045
3,0.031330,0.616620,0.352050
4,0.022746,0.744181,0.233073


In [30]:
subm = pd.merge(test_df[['listing_id']].reset_index(), pred_x.reset_index(), left_index=True, right_index=True)

In [31]:
subm = subm[['listing_id', 'high', 'medium', 'low']]

In [32]:
subm.shape
# (74659, 4)

(74659, 4)

In [33]:
subm.to_csv('Submission_RandomForest_auto120_tune+60feats_desc.csv', index=None)

In [51]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=0.3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False, random_state=21,
            verbose=0, warm_start=False)